# Collaborative Filtering for Anime recommendation
Most of this notebook is copied from this [kaggle notebook](https://www.kaggle.com/code/benroshan/content-collaborative-anime-recommendation) by BEN ROSHAN

In [1]:
## Database 
import warnings
import pandas as pd
import numpy as np

#Visualization 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as pylab
%matplotlib inline
pd.set_option('display.max_columns', 500)
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12, 8

#We use KNN again
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

#ignore warnings
warnings.filterwarnings('ignore')


In [2]:
# We restrict the dataset to avoid running out of memory
anime_data = pd.read_csv('../data/raw/anime_with_synopsis.csv',nrows=10000)
rating_data = pd.read_csv('../data/raw/rating_complete.csv', nrows=1000000)

In [3]:
print(f"""Number of total ratings: {rating_data.shape[0]}. 
Number of different users: {rating_data.user_id.nunique()}. 
Number of different animes: {rating_data.anime_id.nunique()}.""")


Number of total ratings: 1000000. 
Number of different users: 5616. 
Number of different animes: 12155.


In [4]:
anime_data = anime_data.rename(
    columns={'MAL_ID': 'anime_id', 'Name': 'anime_title', 'Score': 'user_rating'})
anime_fulldata = pd.merge(anime_data, rating_data,
                          on='anime_id', suffixes=['', '_user'])

# Creating a dataframe for rating counts
combine_anime_rating = anime_fulldata.dropna(axis=0, subset=['anime_title'])
anime_ratingCount = (combine_anime_rating.
                     groupby(by=['anime_title'])['user_rating'].
                     count().
                     reset_index().rename(
                         columns={'rating': 'totalRatingCount'})
                     [['anime_title', 'user_rating']]
                     )

anime_fulldata = anime_fulldata.merge(anime_ratingCount, left_on = 'anime_title', right_on = 'anime_title', how = 'left')
anime_fulldata = anime_fulldata.rename(columns={'user_rating_x': 'user_rating', 'user_rating_y': 'totalratingcount'})                

anime_feature = anime_fulldata.copy()
anime_feature["user_rating"].replace({-1: np.nan}, inplace=True)

# We discard all the users who have less than 200 votes
counts = anime_feature['user_id'].value_counts()
anime_feature = anime_feature[anime_feature['user_id'].isin(
    counts[counts >= 200].index)]


In [5]:
anime_pivot = anime_feature.pivot_table(
    index='anime_title', columns='user_id', values='rating').fillna(0)
anime_pivot.head()


user_id,3,6,17,19,21,42,44,46,47,52,53,64,73,85,90,110,111,112,121,124,145,146,147,153,155,156,160,172,173,174,181,184,198,204,205,209,214,227,240,243,248,251,257,262,272,284,285,286,290,291,293,310,313,314,320,321,325,326,330,332,336,340,345,346,349,350,360,366,368,371,372,375,377,381,386,389,406,423,427,428,431,432,433,436,437,440,442,444,445,446,457,459,464,469,478,479,481,484,493,498,516,517,524,526,531,542,563,564,566,569,571,577,590,592,596,601,603,608,613,620,631,633,636,639,644,649,652,655,656,662,664,673,674,675,681,689,694,695,699,711,713,716,719,724,725,728,730,745,746,760,764,770,774,776,778,779,781,783,785,792,802,808,819,820,823,836,842,847,848,849,851,853,859,860,864,866,871,876,877,890,894,899,906,911,912,917,921,925,927,932,934,936,937,943,948,950,965,970,974,976,985,989,995,998,1004,1008,1010,1021,1025,1027,1031,1036,1043,1045,1046,1047,1048,1049,1060,1063,1066,1073,1079,1096,1097,1101,1107,1113,1115,1117,1119,1139,1150,1162,1170,1177,1184,1187,1197,1199,1205,1213,1220,1227,1231,1233,1234,1235,1239,1242,...,5211,5213,5224,5226,5230,5231,5234,5236,5240,5241,5242,5250,5251,5256,5257,5260,5263,5264,5266,5267,5274,5281,5285,5293,5294,5302,5304,5321,5322,5323,5324,5326,5335,5344,5345,5346,5347,5359,5360,5368,5375,5386,5391,5393,5394,5401,5407,5409,5418,5424,5434,5435,5452,5455,5470,5472,5473,5479,5480,5483,5493,5496,5508,5520,5523,5525,5528,5534,5538,5539,5543,5549,5550,5552,5557,5566,5568,5569,5570,5579,5580,5581,5586,5588,5589,5595,5597,5606,5611,5612,5614,5623,5626,5640,5644,5646,5647,5648,5651,5653,5655,5658,5659,5660,5661,5662,5672,5677,5686,5687,5692,5696,5700,5702,5705,5707,5712,5713,5717,5726,5727,5735,5739,5740,5744,5747,5758,5762,5763,5767,5769,5771,5776,5784,5788,5790,5801,5809,5813,5817,5819,5824,5828,5834,5846,5852,5857,5861,5866,5871,5872,5874,5880,5885,5889,5892,5899,5903,5905,5910,5912,5929,5936,5947,5955,5956,5962,5964,5982,5983,5993,6000,6008,6023,6032,6033,6037,6044,6051,6053,6054,6060,6071,6082,6083,6084,6086,6088,6099,6101,6106,6107,6110,6111,6130,6133,6139,6143,6145,6148,6149,6152,6159,6160,6163,6170,6174,6181,6182,6191,6197,6198,6204,6206,6211,6214,6223,6224,6238,6243,6246,6249,6252,6253,6254,6266,6273,6284,6286,6298,6299,6309,6313,6316,6321,6327,6334,6356,6357,6360,6364,6371,6373,6382,6383,6387,6399,6416,6417,6418
anime_title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [6]:
anime_matrix = csr_matrix(anime_pivot.values)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(anime_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [7]:
title = "Tokyo Ghoul"
distances, indices = model_knn.kneighbors(
    anime_pivot[anime_pivot.index == title].values.reshape(1, -1), n_neighbors=6)


In [8]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(
            title))
    else:
        print('{0}: {1}, with distance of {2}:'.format(
            i, anime_pivot.index[indices.flatten()[i]], distances.flatten()[i]))


Recommendations for Tokyo Ghoul:

1: Tokyo Ghoul √A, with distance of 0.12358460773285485:
2: Shingeki no Kyojin, with distance of 0.18144884792558824:
3: Noragami, with distance of 0.20813484178510488:
4: No Game No Life, with distance of 0.21724683523292732:
5: Sword Art Online, with distance of 0.22486501022841643:
